In [33]:
import sys
sys.path.append("../")

import numpy as np
from scipy.constants import c, e
import matplotlib.pyplot as plt


import PyCERNmachines.CERNmachines as m
from PySUSSIX import Sussix

### tune analysis

In [ ]:
def tune_analysis(bunch, machine, x_i, xp_i, y_i, yp_i):
		n_turns = x_i.shape[1]

		qx_i = np.empty(bunch.macroparticlenumber)
		qy_i = np.empty(bunch.macroparticlenumber)

		spectrum_x = np.abs(np.fft.fft(x_i, axis = 1))[:,:n_turns/2]
		tune_peak_x = np.float_(np.argmax(spectrum_x, axis=1))/float(n_turns)

		spectrum_y = np.abs(np.fft.fft(y_i, axis = 1))[:,:n_turns/2]
		tune_peak_y = np.float_(np.argmax(spectrum_y, axis=1))/float(n_turns)

		print 'analysing particle spectra ... this may take some time.'
		for p_idx in xrange(bunch.macroparticlenumber):
			
			SX = Sussix()
			SX.sussix_inp(nt1=1, nt2=n_turns, idam=2, ir=0, tunex=tune_peak_x[p_idx], tuney=tune_peak_y[p_idx])

			SX.sussix(x_i[p_idx,:], xp_i[p_idx,:],
					  y_i[p_idx,:], yp_i[p_idx,:],
					  x_i[p_idx,:], xp_i[p_idx,:]) 
			qx_i[p_idx] = SX.ox[0]
			qy_i[p_idx] = SX.oy[0]
			
			sys.stdout.write('\rparticle %d'%p_idx)
				
		x_centroid = np.mean(x_i, axis=0)
		xp_centroid = np.mean(xp_i, axis=0)
		y_centroid = np.mean(y_i, axis=0)
		yp_centroid = np.mean(yp_i, axis=0)
		
		#print x_centroid.shape

		SX = Sussix()
		SX.sussix_inp(nt1=1, nt2=n_turns, idam=2, ir=0, tunex=machine.Q_x%1, tuney=machine.Q_y%1)
		SX.sussix(x_centroid, xp_centroid,
				  y_centroid, yp_centroid,
				  x_centroid, xp_centroid) 
		qx_centroid = SX.ox[0]
		qy_centroid = SX.oy[0]

		return qx_i, qy_i, qx_centroid, qy_centroid
    

def plot_tune_spread(bunch, machine, x_i, xp_i, y_i, yp_i):
    qx_i, qx_i, qx_centroid, qy_centroid = tune_analysis(bunch,  machine, x_i, xp_i, y_i, yp_i)
    plt.plot(np.abs(qx_i), np.abs(qy_i), '.')
    plt.plot([np.modf(machine.Q_x)[0]], [np.modf(machine.Q_y)[0]], 'ro', label='Bare tune')
    #if flag_initial_displacement:
    #    plt.plot([np.abs(qx_centroid)], [np.abs(qy_centroid)], 'xg', markersize = 10, label='Coherent tune')
    #plt.plot([.15, .30], [.15, .30], 'k')
    plt.legend(loc='lower right')
    plt.xlabel('$Q_x$');plt.ylabel('$Q_y$')
    plt.axis('equal')
    plt.show()

### simulation parameters

In [46]:
n_particles = 10000#1000000
n_turns = 128
epsn_x = 2.5e-6
epsn_y = 2.5e-6
sigma_z = 0.23
intensity = 1.0e11

### load optics (sps)

In [47]:
optix = np.loadtxt('SPS_optics_q20.dat')
beta_x = optix[:,0]
beta_y = optix[:,1]
D_x = optix[:,2]
D_y = optix[:,3]
n_segments = optix.shape[0]
print n_segments

36


### define the machine (sps)

In [48]:

machine = m.SPS(n_segments=n_segments, machine_configuration='Q20-injection', longitudinal_focusing='linear',
               beta_x=beta_x, beta_y=beta_y, D_x=D_x, D_y=D_y)

Synchrotron init. From kwargs: D_x = array([ 7.95719698,  5.65643658,  1.87130559,  1.2283289 ,  4.51340999,
        7.7112296 ,  6.91312723,  3.09461482, -0.45438555, -1.00827514,
       -0.34154528,  3.43066005,  7.1540663 ,  7.56256528,  4.15685308,
        1.09969507,  2.12782259,  5.98456643,  7.95719698,  5.65643658,
        1.87130559,  1.2283289 ,  4.51340999,  7.7112296 ,  6.91312723,
        3.09461482, -0.45438555, -1.00827514, -0.34154528,  3.43066005,
        7.1540663 ,  7.56256528,  4.15685308,  1.09969507,  2.12782259,
        5.98456643])
Synchrotron init. From kwargs: D_y = array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
Synchrotron init. From kwargs: beta_y = array([ 32.24200162,  32.00695787,  32.3821595 ,  32.09037677,
        32.12487183,  32.3682195 ,  31.99203617,  32.2768934 ,
        32.76208419,  32.69075339

### create gaussian bunch

In [49]:
bunch = machine.generate_6D_Gaussian_bunch(n_macroparticles=n_particles, intensity=intensity,
                                          epsn_x=epsn_x, epsn_y=epsn_y, sigma_z=sigma_z)

### create the maps

In [50]:
# particle positions for tune analysis
x_i = np.empty((n_particles, n_turns))
xp_i = np.empty((n_particles, n_turns))
y_i = np.empty((n_particles, n_turns))
yp_i = np.empty((n_particles, n_turns))

# track and store
for i in range(n_turns):    
    machine.track(bunch)
    
    sys.stdout.write('\rturn %d'%i)
    sys.stdout.flush()
    
    x_i[:,i] = bunch.x[:]
    xp_i[:,i] = bunch.xp[:]
    y_i[:,i] = bunch.y[:]
    yp_i[:,i] = bunch.yp[:]
print '\nDONE'


turn 127
DONE


In [ ]:
samples = 2000
plot_tune_spread(bunch, machine, x_i[1:samples,:], xp_i[1:samples,:], y_i[1:samples,:], yp_i[1:samples,:])

analysing particle spectra ... this may take some time.
particle 6214